# Training Models

In  this  chapter  we  will  start  by  looking  at  the  Linear  Regression  model,  one  of  the simplest models there is. We will discuss two very different ways to train it:

- Using a direct “closed-form” equation that directly computes the model parameters  that  best  fit  the  model  to  the  training  set  (i.e.,  the  model  parameters  that minimize the cost function over the training set).

- Using  an  iterative  optimization  approach  called  Gradient  Descent  (GD)  that gradually  tweaks  the  model  parameters  to  minimize  the  cost  function  over  thetraining  set,  eventually  converging  to  the  same  set  of  parameters  as  the  firstmethod. We will look at a few variants of Gradient Descent that we will use againand again when we study neural networks: Batch GD, Mini-batch GD,and Stochastic GD.

Next we will look at Polynomial Regression, a more complex model that can fit non‐linear  datasets.  Since  this  model  has  more  parameters  than  Linear  Regression,  it  is more prone to overfitting the training data, so we will look at how to detect whether or not this is the case using learning curves, and then we will look at several regularization techniques that can reduce the risk of overfitting the training set. Finally,  we  will  look  at  two  more  models  that  are  commonly  used  for  classification tasks: 

- Logistic Regression and 
- Softmax Regression.

## Linear Regression

